In [1]:
# Import libraries

import psycopg2
import pandas as pd

In [2]:
# Connect to postgree
conn = psycopg2.connect(
    host="localhost",
    port=5433,
    database="Library_Management",
    user="postgres",
    password="postgres"
)

In [3]:
# 1. Berapa total judul dan category buku yang tersedia

no_1="""
select 
    count(distinct(book_title)) as jumlah_judul_buku,
    count(distinct(category)) as jumlah_category_buku
from books
"""

df_1 = pd.read_sql(no_1, conn)
df_1

C:\Users\User\AppData\Local\Temp\ipykernel_12816\3102326642.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_1 = pd.read_sql(no_1, conn)


,jumlah_judul_buku,jumlah_category_buku
0,33,10


In [4]:
# 2. Berapa total buku yang disewa dan telah dikembalikan setiap branch

no_2="""
select
	branch,
	total_dipinjam,
	total_dibalikin,
	(total_dibalikin::numeric/nullif(total_dipinjam,0)) as ratio
from
	(select 
		b.branch_id as branch,
		count(i.issued_id) as total_dipinjam,
		count(rs.return_id) as total_dibalikin
	from issue as i
	join employee as e
		on e.emp_id=i.issued_emp_id
	join branch as b
		on b.branch_id=e.branch_id
	left join return_stat as rs
		on rs.issued_id=i.issued_id
	group by 1)	
order by 4 desc
"""
df_2=pd.read_sql(no_2, conn)
df_2

C:\Users\User\AppData\Local\Temp\ipykernel_12816\1349816529.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2=pd.read_sql(no_2, conn)


,branch,total_dipinjam,total_dibalikin,ratio
0,B004,4,3,0.750000
1,B001,17,10,0.588235
2,B005,9,3,0.333333
3,B002,2,0,0.000000
4,B003,2,0,0.000000


In [5]:
# 3. Berapa total penyewa buku dan alamatnya (member)
no_3="""
select 
	count(members_id) as jumlah_penyewa_buku,
	count (members_addres) as jumlah_asal_penyewa
from members
"""
df_3=pd.read_sql(no_3, conn)
df_3

C:\Users\User\AppData\Local\Temp\ipykernel_12816\4145275418.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3=pd.read_sql(no_3, conn)


,jumlah_penyewa_buku,jumlah_asal_penyewa
0,12,12


In [6]:
# 4. Berikan top penyewa buku (member)

no_4="""
select 
	m.members_name,
	m.members_id,
	count(i.issued_id) as jumlah_buku_disewa
from issue as i
join members as m
	on m.members_id = i.issued_member_id
group by 1,2
order by jumlah_buku_disewa desc
"""
df_4=pd.read_sql(no_4, conn)
df_4

C:\Users\User\AppData\Local\Temp\ipykernel_12816\1483446202.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4=pd.read_sql(no_4, conn)


,members_name,members_id,jumlah_buku_disewa
0,Ivy Martinez,C109,7
1,Grace Taylor,C107,6
2,Jack Wilson,C110,6
3,Eve Brown,C105,5
4,Frank Thomas,C106,4
5,Henry Anderson,C108,2
6,Dave Wilson,C104,1
7,Alice Johnson,C101,1
8,Carol Davis,C103,1
9,Bob Smith,C102,1


In [7]:
# 5. Berikan daerah paling banyak menyewa buku

no_5="""
select
	m.members_addres,
	count(i.issued_id) as total_peminjaman
from issue as i
join members as m
	on m.members_id = i.issued_member_id
group by 1
order by total_peminjaman desc
"""
df_5=pd.read_sql(no_5, conn)
df_5

C:\Users\User\AppData\Local\Temp\ipykernel_12816\2247482818.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5=pd.read_sql(no_5, conn)


,members_addres,total_peminjaman
0,567 Oak St,7
1,345 Walnut St,6
2,678 Pine St,6
3,890 Maple St,5
4,234 Cedar St,4
5,456 Birch St,2
6,123 Main St,1
7,456 Elm St,1
8,125 Oak St,1
9,567 Pine St,1


In [8]:
# 6. Berikan branch paling banyak menyewakan buku

no_6="""
select
	b.branch_id,
	b.branch_address,
	count(distinct(i.issued_id)) as total_disewa
from issue as i
join employee as e
	on e.emp_id = i.issued_emp_id
join branch as b
	on b.branch_id =e.branch_id
group by 1
order by 3 desc
"""
df_6=pd.read_sql(no_6, conn)
df_6

C:\Users\User\AppData\Local\Temp\ipykernel_12816\3189159512.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6=pd.read_sql(no_6, conn)


,branch_id,branch_address,total_disewa
0,B001,123 Main St,17
1,B005,890 Maple St,9
2,B004,567 Pine St,4
3,B002,456 Elm St,2
4,B003,789 Oak St,2


In [9]:
# 7. Berikan buku yang paling banyak di sewa member dan harganya

no_7="""
select
	book_title,
	book_isbn,
	book_category,
	total_rent,
	price,
	total,
	sum(total) over () as revenue
from
	(select 
		judul as book_title,
		isbn as book_isbn,
		category as book_category,
		total_peminjaman as total_rent,
		harga as price,
		total_peminjaman*harga as total
	from
		(select 
			b.book_title as judul,
			b.isbn as isbn,
			b.category as category,
			count(i.issued_id) as total_peminjaman,
			b.rental_price as harga
		from issue as i
		join books as b
			on b.isbn=i.issued_book_isbn
		group by 1,2
		order by 4 desc))
"""
df_7=pd.read_sql(no_7, conn)
df_7

C:\Users\User\AppData\Local\Temp\ipykernel_12816\2693330477.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_7=pd.read_sql(no_7, conn)


,book_title,book_isbn,book_category,total_rent,price,total,revenue
0,Harry Potter and the Sorcerers Stone,978-0-679-76489-8,Fantasy,2,7.0,14.0,214.0
1,Animal Farm,978-0-330-25864-8,Classic,2,5.5,11.0,214.0
2,The Great Gatsby,978-0-525-47535-5,Classic,2,8.0,16.0,214.0
3,The Road,978-0-670-81302-4,Dystopian,1,7.0,7.0,214.0
4,The Histories,978-0-14-044930-3,History,1,5.5,5.5,214.0
5,A Game of Thrones,978-0-09-957807-9,Fantasy,1,7.5,7.5,214.0
6,Dune,978-0-345-39180-3,Science Fiction,1,8.5,8.5,214.0
7,1491: New Revelations of the Americas Before C...,978-0-7432-7357-1,History,1,6.5,6.5,214.0
8,Jane Eyre,978-0-141-44171-6,Classic,1,4.0,4.0,214.0
9,The Catcher in the Rye,978-0-553-29698-2,Classic,1,7.0,7.0,214.0


In [10]:
# 8. Buat table baru yang berisi data buku yang belum dikembalikan

no_8="""
drop table if exists buku_belum_balik ;
create table buku_belum_balik as
select
	isbn,
	judul_buku,
	nama,
	m_id
from
	(select
		m.members_name as nama,
		m.members_id as m_id,
		bk.book_title as judul_buku,
		bk.isbn as isbn,
		rs.return_date as pengembalian,
		i.issued_date as peminjaman
	from issue as i
	join members as m
		on m.members_id=i.issued_member_id
	join books as bk
		on bk.isbn=i.issued_book_isbn
	left join return_stat as rs
		on rs.issued_id =i.issued_id
		group by 1,2,3,4,5,6)
where pengembalian is null;
select * from buku_belum_balik
"""
df_8=pd.read_sql(no_8, conn)
df_8

C:\Users\User\AppData\Local\Temp\ipykernel_12816\2654085189.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_8=pd.read_sql(no_8, conn)


,isbn,judul_buku,nama,m_id
0,978-0-451-52993-5,Fahrenheit 451,Bob Smith,C102
1,978-0-345-39180-3,Dune,Carol Davis,C103
2,978-0-06-025492-6,Where the Wild Things Are,Dave Wilson,C104
3,978-0-06-112241-5,The Kite Runner,Eve Brown,C105
4,978-0-06-440055-8,Charlotte's Web,Eve Brown,C105
5,978-0-14-027526-3,A Tale of Two Cities,Eve Brown,C105
6,978-0-679-77644-3,Beloved,Eve Brown,C105
7,978-0-7434-7679-3,The Stand,Eve Brown,C105
8,978-0-06-112008-4,To Kill a Mockingbird,Frank Thomas,C106
9,978-0-451-52994-2,Moby Dick,Frank Thomas,C106


In [11]:
# 9. Tampilkan waktu peminjaman buku

no_9="""
select
	i.issued_id,
	(rs.return_date-i.issued_date) as waktu_peminjaman
from issue as i
left join return_stat as rs
	on rs.issued_id=i.issued_id
where rs.return_date is not null
order by 2 
"""
df_9=pd.read_sql(no_9, conn)
df_9

C:\Users\User\AppData\Local\Temp\ipykernel_12816\2522681522.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_9=pd.read_sql(no_9, conn)


,issued_id,waktu_peminjaman
0,IS106,52
1,IS107,53
2,IS108,54
3,IS109,55
4,IS110,56
5,IS111,57
6,IS112,58
7,IS113,59
8,IS114,60
9,IS115,61


In [12]:
# 10. Tampikan total revenue tiap branch

no_10="""
select 
	b.branch_id,
	count(issued_id)as total_buku_dipinjam,
	sum(bk.rental_price)as revenue
from issue as i
join employee as e
	on e.emp_id = i.issued_emp_id
join branch as b
	on b.branch_id = e.branch_id
join books as bk
	on bk.isbn=i.issued_book_isbn
group by 1
order by 3 desc
"""
df_10=pd.read_sql(no_10, conn)
df_10

C:\Users\User\AppData\Local\Temp\ipykernel_12816\2160154490.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_10=pd.read_sql(no_10, conn)


,branch_id,total_buku_dipinjam,revenue
0,B001,17,111.5
1,B005,9,50.0
2,B004,4,26.5
3,B003,2,14.0
4,B002,2,12.0
